# Assignment 5 - Issue #854

Fernando Mendoza | Adrea Pezo | Michel Cotrina | Estefanny Gil | Armando Ore

The script should not give an error. Any mistake will be consider as 0.
<br>We want an excel file with all the available presidential elections in this webpage.
https://infogob.jne.gob.pe/Eleccion
<br>The final excel should have these 3 columns 
(Elecciones | ORGANIZACIÓN POLÍTICA | TOTAL VOTOS)

In [45]:
#!pip install selenium
#!pip install webdriver-manager

## a. Setup for Chrome only

In [83]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time 
import pandas as pd

# Path for google Chrome
chrome_driver_path = r'chromedriver-win64/chromedriver.exe'
# Configue the service for google chrome
service = Service(executable_path=chrome_driver_path)
# Start the WebDriver for google chrome
driver = webdriver.Chrome(service=service)

## b. Setup for FireFox only

In [86]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time 
import pandas as pd

# Path for geckodriver (Firefox)
gecko_driver_path = r'geckodriver-v0.34.0-win64\geckodriver.exe'

# Configure the service for Firefox
service = FirefoxService(executable_path=gecko_driver_path)

# Start the WebDriver for Firefox
driver = webdriver.Firefox(service=service)


## c. Scrip for "https://infogob.jne.gob.pe/Eleccion"

In [86]:
# Maximize window
driver.maximize_window()

# Base URL for the web scraping. Open a new window
url_base = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url_base)


In [88]:
# Display "Tipo de proceso" options
opt_Presidenciales = driver.find_element(By.XPATH,
                                         '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[1]')
opt_Presidenciales.click()
time.sleep(3)  # Waits 3 seconds; increase if internet connection is slow

# Select "ELECCIONES PRESIDENCIALES"
opt_Presidenciales_2 = driver.find_element(By.XPATH, 
                                           '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
opt_Presidenciales_2.click()

time.sleep(3)  # Waits 3 seconds; increase if internet connection is slow

In [90]:
# Clicks on the dropdown list to display the available presidential elections
driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div').click()
time.sleep(3)  # Waits 3 seconds; increase if internet connection is slow

# Getting the list of elections
items = driver.find_elements(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div/div[2]/div')

# Extract the text and split it into a list by line breaks
elecciones = []
for item in items:
    elecciones.extend(item.text.split('\n'))

# Remove the first element if it's irrelevant ([SELECCIONE])
elecciones = elecciones[1:]

# Join the elections with newline characters and print them
elecciones_str = "\n".join(elecciones)
print(elecciones_str)

PRESIDENCIAL 2021 - 2DA VUELTA
PRESIDENCIAL 2021
PRESIDENCIAL 2016 - 2DA VUELTA
PRESIDENCIAL 2016
PRESIDENCIAL 2011 - 2DA VUELTA
PRESIDENCIAL 2011
PRESIDENCIAL 2006 - 2DA VUELTA
PRESIDENCIAL 2006
PRESIDENCIAL 2001 - 2DA VUELTA
PRESIDENCIAL 2001
PRESIDENCIAL 2000 - 2DA VUELTA
PRESIDENCIAL 2000
PRESIDENCIAL 1995
PRESIDENCIAL 1990 - 2DA VUELTA
PRESIDENCIAL 1990
PRESIDENCIAL 1985
PRESIDENCIAL 1980
PRESIDENCIAL 1963
PRESIDENCIAL 1962
PRESIDENCIAL 1956
PRESIDENCIAL 1950
PRESIDENCIAL 1945
PRESIDENCIAL 1939
PRESIDENCIAL 1936
PRESIDENCIAL 1931


## 3. Iteration

In [87]:
# Maximize window
driver.maximize_window()

# Base URL for the web scraping. Open a new window
url_base = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url_base)

In [90]:
dataframes_list = []

url = 'https://infogob.jne.gob.pe/Eleccion'

for number in range(2,27):
    driver.get(url)
    dropdown = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    dropdown.click()
    time.sleep(1)
    
    presidenciales = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidenciales.click()
    time.sleep(1)

    eleccion=driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    eleccion.click()
    time.sleep(1)

    election_1 = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(number))#de 2 a 26
    election_1.click()
    eleccion_name= election_1.text
    time.sleep(1)

    entrar = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    entrar.click()
    time.sleep(2)

    resultado=driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultado.click()
    time.sleep(1)

    table_path=driver.find_element(By.ID,'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute( 'innerHTML' )

    #We create headers to group the information 
    #Add hmtl text, so there are no problems with the read.html
    head="<table id=\"gridEleccionResultadosElectorales\" class=\"table table-hover table-striped\">"
    table_new = head + table_html[21:-1]
    table_new
    table = pd.read_html(tabla_new)
    table_clean=table[ 0 ].iloc[ : , : ]
    table_clean = table_clean.loc[:, ['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]
    table_clean.insert(0, 'Elecciones', eleccion_name)
    dataframes_list.append(table_clean)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18504\2797879098.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(tabla_new)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18504\2797879098.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(tabla_new)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18504\2797879098.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(tabla_new)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18504\2797879098.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read f

In [100]:
#Transform the list of dataframes into a single dataframe
    
df_vote = pd.concat(dataframes_list, ignore_index=True)

df_vote

,Elecciones,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,,PARTIDO POLITICO NACIONAL PERU LIBRE,8836380
1,,FUERZA POPULAR,8792117
2,,VOTOS EN BLANCO,121489
3,,VOTOS NULOS,1106816
4,,PARTIDO POLITICO NACIONAL PERU LIBRE,2724752
...,...,...,...
152,,PARTIDO REPUBLICANO,30803
153,,UNION REVOLUCIONARIA,152149
154,,PARTIDO APRISTA PERUANO,106088
155,,PARTIDO DESCENTRALISTA,21950


In [102]:
df_vote.to_excel( r'group_7_ass_5_2024_2.xlsx' , index = False )